In [2]:
!cp -r "/kaggle/input/sign-language/data" /kaggle/working/

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import os 
no_sequences = 100 # Number of videos per action
sequence_length = 30 # Length of each video sequence: 30 frames
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('/kaggle/working/data')

actions = np.array(['hello', 'thanks', 'iloveyou', 'yes', 'no', 'please', 'sorry','goodbye',
                    'stop', 'wait', 'go', 'come', 'love', 'help', 'like', 'dislike', 
                    'happy', 'sad', 'I', 'you', 'it'])

label_map = {label : num for num, label in enumerate(actions)}
print(label_map)

{'hello': 0, 'thanks': 1, 'iloveyou': 2, 'yes': 3, 'no': 4, 'please': 5, 'sorry': 6, 'goodbye': 7, 'stop': 8, 'wait': 9, 'go': 10, 'come': 11, 'love': 12, 'help': 13, 'like': 14, 'dislike': 15, 'happy': 16, 'sad': 17, 'I': 18, 'you': 19, 'it': 20}


In [6]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            file_path = os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy")
            keypoints = np.load(file_path)

            # Chỉ lấy right hand và left hand (cuối mảng)
            lh_rh = keypoints[-(21*3*2):]  # 21*3 left + 21*3 right
            window.append(lh_rh)

        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)
#print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [7]:
%load_ext tensorboard
log_dir = 'logs'
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 126)))  # 126 = 21*3*2
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Train model
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=64)  

model.save('best_model.h5') 

I0000 00:00:1750521814.698505      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1750521814.699199      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100


I0000 00:00:1750521823.519862     101 service.cc:148] XLA service 0x7dacec111970 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750521823.521227     101 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1750521823.521282     101 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1750521824.399194     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


 7/32 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - categorical_accuracy: 0.0252 - loss: 3.0489

I0000 00:00:1750521827.562435     101 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 ━━━━━━━━━━━━━━━━━━━━ 17s 204ms/step - categorical_accuracy: 0.0550 - loss: 3.0312 - val_categorical_accuracy: 0.2095 - val_loss: 2.7950
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - categorical_accuracy: 0.1875 - loss: 2.8462 - val_categorical_accuracy: 0.1333 - val_loss: 2.8278
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - categorical_accuracy: 0.1788 - loss: 2.6896 - val_categorical_accuracy: 0.2381 - val_loss: 2.4260
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - categorical_accuracy: 0.2621 - loss: 2.1786 - val_categorical_accuracy: 0.3238 - val_loss: 2.1502
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - categorical_accuracy: 0.3613 - loss: 1.8310 - val_categorical_accuracy: 0.4571 - val_loss: 1.6572
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - categorical_accuracy: 0.4384 - loss: 1.5581 - val_categorical_accuracy: 0.3238 - val_loss: 1.6525
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - categorical_accuracy: 0.4829 - loss: 1.3837 